## 학습 전제
numpy, pandas 활용 >> 표 다루기   
pandas 문법, 다양한 메서드 활용 >> 코드 이해 가능  
matplotlib 활용 데이터 시각화 >> 코드 이해 가능  
데이터셋 >> train/test 데이터셋으로 나누어 모델 학습 및 검증

## 학습 목표  

다양한 피처의 데이터셋을 믿바닥부터 샅샅이 뜯어보고, 포켓몬 분류를 위한 피처를 생각해보기  
모델 학습 시작 전 모든 컬럼에 대해 >> 그래프 시각화, 피벗 테이블 등 활용 >> 다양한 방법으로 충분한 EDA 진행  
모델 학습에 넣기 위해 전처리 필요한 범주형/문자열 데이터에 대한 전처리 >> 원-핫 인코딩 등 적절하게 진행  
전체 데이터셋 train/test 데이터셋으로 나누고 >> 절절한 분류 모델(Decision Tree)를 선택해 학습시키고 베이스라인과 비교  

#### *EDA (Exploratory Data Analysis), 탐색적 데이터 분석. 더 좋은 데이터 분석과 더 좋은 머신러닝을 만들기 위해 필수적인 과정 

EDA 관련 블로그 https://www.insilicogen.com/blog/361

### <span style="color:red"> POINT! </span>

데이터를 다루기 전, 데이터셋을 한번이라고 이해하고 시작 

In [2]:
#라이브러리 가져오기

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
print('슝=3')

슝=3


* numpy와 pandas는 1차원 또는 2차원 형식의 표 데이터를 다루기에 최적화  
* seaborn과 matplotlib은 데이터를 그래프 등으로 시각화  
* seaborn은 matplotlib의 상위 버전. matplotlib이 조금 더 단순하지만 raw , seaborn은 보다 고급화된 그래프 표현 

In [12]:
#데이터셋 pandas로 불러오기

csv_path = "/Users/user/CatchNode/data/Pokemon.csv"
original_data = pd.read_csv(csv_path)
print('슝=3')


슝=3


In [13]:
#데이터 꺼내기

pokemon = original_data.copy()
print(pokemon.shape)
pokemon.head()

(800, 13)


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


데이터셋 구성 800행, 13열
포켓몬 800마리 각 포켓몬을 설명하는 특성 13개

우리가 타겟으로 두고 확인할 데이터는 Legendary이므로, Legendary == True 값을 가지는 레전드 포켓몬 데이터셋은 legendaryt 변수에, Legendary == False 값을 가지는 일반 포켓몬 데이터셋은 ordinary 변수에 저장 

In [15]:
# 전설의 포켓몬 데이터셋

legendary = pokemon[pokemon["Legendary"] == True].reset_index(drop=True)
print(legendary.shape)
legendary.head()

(65, 13)


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,144,Articuno,Ice,Flying,580,90,85,100,95,125,85,1,True
1,145,Zapdos,Electric,Flying,580,90,90,85,125,90,100,1,True
2,146,Moltres,Fire,Flying,580,90,100,90,125,85,90,1,True
3,150,Mewtwo,Psychic,NaN,680,106,110,90,154,90,130,1,True
4,150,MewtwoMega Mewtwo X,Psychic,Fighting,780,106,190,100,154,100,130,1,True


In [18]:
# 일반 포켓몬 데이터셋

ordinary = pokemon[pokemon["Legendary"] == False].reset_index(drop=True)
print(ordinary.shape)
ordinary.head()

(735, 13)


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


### <span style="color:red"> POINT! </span>

데이터 다루기 전 가장 기본적으로 빈 데이터 (결측치) ! 먼저 확인

In [17]:
pokemon.isnull().sum()

#               0
Name            0
Type 1          0
Type 2        386
Total           0
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

데이터셋의 성격에 따라 빈 데이터를 어떻게 다루어야 할지에 대한 방법이 달라짐

In [19]:
#전체 컬럼 출력 
print(len(pokemon.columns))
pokemon.columns

13


Index(['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

In [20]:
# 첫 번째 칼럼 ID 칼럼 #
len(set(pokemon["#"]))

721

전체 데이터는 800 #데이터는 721이기 때문에 같은 # 값을 가지는 포켓몬들이 있다는 것을 확인할 수 있다. 

In [21]:
pokemon[pokemon["#"] == 6]

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False


In [22]:
# 이름, 문자열로 나타는 데이터 
# 모든 이름이 유일한 이름인지, 중복된 이름은 없는지 확인.
# 다음과 같이 pokemon['Name']을 집합(set)으로 만들어준 후 길이(len)를 확인하면 중복이 사라지면서 유일한 이름의 개수 확인
len(set(pokemon["Name"]))

800

모든 이름 unique 하다는 것을 확인